# <center>Spatial Join with ArcGIS
<img src="https://github.com/Ray800413/Spatial_Join_with_ArcGIS/blob/master/image/spatial_join.png?raw=true" style="width:800px;">

## 1. Introduction
This jupyter notebook is the training tutorial for the use of the `Spatial Join` tool. This training tutorial assumes you already have the intermediate level of knowledge of the **_Spatial Join_** and the basic understanding of **_FieldMaps_**. It also assumes that you are familiar with other geospatial tools, such as the `Select Layers by Attribute` and `Clip` tool. In order to complete this training tutorial smoothly, except for the skills of using `Arcpy` scripts, you should have basic understanding of the libaries coming along with `Python`, such as `pandas` and `regex`, as well.

For this tutorial, we will use two data: **(1) Car accident data in Minnesota in 2016 (named "mn-2016-acc.csv") (2) Minnesota County shapefile (named "County_Boundaries_in_Minnesota" in "shp_bdry_counties" folder)** to answer some statistical questions such as identify which county has the highest fatal number caused by accidents and what type of accident most frequently occured in each county.

## 2. Workflow

### 2.1 Data Munging

Before we get started to analyze the data, we often need to perform some cleanup tasks to ensure that it is in a good shape that we are able to work with.

#### 2.1.1 Working with pandas

`pandas` is a useful python module that can help you facilitate data munging. In the cells below, we will first read our data and then select out the columns that are neeeded for analysis later on. 

For more information about `pandas`, please refer to [this documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)

In [1]:
# Import pandas module
import pandas as pd

In [2]:
# Read the file into dataframe. 
df = pd.read_csv("mn-2016-acc.csv", low_memory = False)
df.head(1)

,ACCN,AGENCY,LOCCASE,HITRUN,PROPDAM,NUMMV,NUMFAT,NUMINJ,DOLMIN,ACCDATE,...,RTSYS,WKZNREL,CITY,XCOORD,YCOORD,TOWNSHIP,ACCTIME,URBRURT,ACCDAY,ACCYEAR
0,80001,3280,BEP16-0101,2,5,2,0,0,1.0,1/8/2016,...,2.0,2,390,43.64485697,-94.09181618,.,1705,7,6,2016


In [3]:
#Let's see all the columns' name.
df.columns

Index(['ACCN', 'AGENCY', 'LOCCASE', 'HITRUN', 'PROPDAM', 'NUMMV', 'NUMFAT',
       'NUMINJ', 'DOLMIN', 'ACCDATE', 'COUNTY', 'CITYTWP', 'CITYNAME',
       'ACCTYPE', 'SBUS', 'LOCFHE', 'BRIDGE', 'WKZNTYPE', 'LOCWKZN', 'WORKERS',
       'RDSURF', 'INTREL', 'WEATHER1', 'WEATHER2', 'LIGHT', 'DIAGRAM',
       'OFFTYPE', 'INTERSECT', 'ACCSEV', 'CFR1', 'CFR2', 'FATAL', 'FATWKZN',
       'INJURY', 'INTYPE', 'LANDOWN', 'LEPRES', 'NUMNM', 'ONROAD', 'RTSYS',
       'WKZNREL', 'CITY', 'XCOORD', 'YCOORD', 'TOWNSHIP', 'ACCTIME', 'URBRURT',
       'ACCDAY', 'ACCYEAR'],
      dtype='object')

In [4]:
# Slice out the columns that include information we need for analysis. 
# ACCTYPE = accident type; FATAL = fatal or non fatal; NUMFAT = fatal number; 
# INJURY = injury or non injury; NUMINJ = injury number; ACCDAY = weekday of accident;
# COUNTY = county of accident; YCOORD = longitude of acciden;, XCOORD = latitude of accident; 
# ACCDATE = date of accident
df1 = df[["ACCTYPE", "FATAL","NUMFAT", "INJURY","NUMINJ", 
          "ACCDAY", "COUNTY", 'XCOORD', 'YCOORD', "ACCDATE"]]

# Print first 5 rows to check it the output is correct
df1.head()

,ACCTYPE,FATAL,NUMFAT,INJURY,NUMINJ,ACCDAY,COUNTY,XCOORD,YCOORD,ACCDATE
0,10,2,0,2,0,6,22,43.64485697,-94.09181618,1/8/2016
1,11,2,0,2,0,6,47,45.31667467,-94.40934696,7/22/2016
2,28,2,0,2,0,2,73,45.55586262,-94.18532244,12/5/2016
3,10,2,0,2,0,2,62,44.94822768,-93.09167463,10/17/2016
4,10,2,0,1,2,5,27,.,.,10/6/2016


In the above output, you can see that cells containing x and y coordinate information have some records missing these values and are represented by `.`. These records are not applicable for usage so the next step is to remove them from our data.

In [5]:
# Slice out the rows do not contain "." in coordinate cells
df2 = df1[df1.XCOORD != "."]
# Check the result
df2.head()

,ACCTYPE,FATAL,NUMFAT,INJURY,NUMINJ,ACCDAY,COUNTY,XCOORD,YCOORD,ACCDATE
0,10,2,0,2,0,6,22,43.64485697,-94.09181618,1/8/2016
1,11,2,0,2,0,6,47,45.31667467,-94.40934696,7/22/2016
2,28,2,0,2,0,2,73,45.55586262,-94.18532244,12/5/2016
3,10,2,0,2,0,2,62,44.94822768,-93.09167463,10/17/2016
5,69,2,0,2,0,6,62,44.93412681,-93.1462896,1/1/2016


Now, we are going to check out the data type of each column to ensure they are in the applicable type. 

In [6]:
# Check columns' datatype
df2.dtypes

ACCTYPE     int64
FATAL       int64
NUMFAT      int64
INJURY      int64
NUMINJ      int64
ACCDAY      int64
COUNTY      int64
XCOORD     object
YCOORD     object
ACCDATE    object
dtype: object

In [7]:
# Convert fields to desired data types
# Convert "County" type will help us slice our needed counties easier
# Coordinate fields need to be converted to float for future usage
df3 = df2.astype({"COUNTY": "str", "XCOORD":"float", "YCOORD": "float"})

# Check the result
df3.dtypes

ACCTYPE      int64
FATAL        int64
NUMFAT       int64
INJURY       int64
NUMINJ       int64
ACCDAY       int64
COUNTY      object
XCOORD     float64
YCOORD     float64
ACCDATE     object
dtype: object

Since in our later analysis, we would like to know which month has the most occurence of accidents. We need to slice month information out from the "ACCDATE" field. To do so, we would take advantage of `regex`.

For more information about `regex`, please refer to [this documentation](https://docs.python.org/3/library/re.html#)

In [8]:
# Import re module
import re

In [9]:
# Create a pattern object to extract month infomation
p = re.compile(r"(\w+)/.+/.+")

# Add a new field to store month information
df3["MONTH"] = df3.ACCDATE.str.extract(p)

# Display the first five rows to check the code executed correctly
df3.head()

,ACCTYPE,FATAL,NUMFAT,INJURY,NUMINJ,ACCDAY,COUNTY,XCOORD,YCOORD,ACCDATE,MONTH
0,10,2,0,2,0,6,22,43.644857,-94.091816,1/8/2016,1
1,11,2,0,2,0,6,47,45.316675,-94.409347,7/22/2016,7
2,28,2,0,2,0,2,73,45.555863,-94.185322,12/5/2016,12
3,10,2,0,2,0,2,62,44.948228,-93.091675,10/17/2016,10
5,69,2,0,2,0,6,62,44.934127,-93.146290,1/1/2016,1


The spatial extent of the accidents data cover the entire state of Minnesota; however, we are only interested in the accidents occured in the Metro city of Minnesota. Therefore, we will extract the accidents records in the Metro city in the following cells. 

In [10]:
# Create a list to store FIP code of seven counties
counties = ["2", "27", "82", "62", "70", "10", "19"]

# Preserve the rows that have the above county code inside
df4 = df3[df3["COUNTY"].isin(counties)]

# Display the COUNTY field information to see if only seven counties are preserved
df4.COUNTY.describe()

count     45958
unique        7
top          27
freq      21857
Name: COUNTY, dtype: object

In [11]:
# Covert data type of MONTH field to integer
df4['MONTH'] = df4['MONTH'].astype("int")

# Check the result
df4["MONTH"].dtypes

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


dtype('int32')

In [12]:
# Save the dataframe to csv file
df4.to_csv("Traffic Accidents in Metro1.csv", index = False)

Excellent!! Right now, we have finished our accidents data cleanup and it is ready to use. Recaping from the above cells, you used `pandas` module to manage and modify your data in several lines of succinct codes and `regext` module to extract needed information in an efficient manner. Imagine the time you need to spend if you want to do these tasks if you working in Excel. They really make our life much easier, right? 

#### 2.1.2 Working with arcpy

In the following cells, we are going to work using `Arcpy`. `ArcPy` is a Python site package that provides a useful and productive way to perform geographic data analysis, data conversion, data management, and map automation with Python developed my Esri. 

For more information about `Arcpy`, please refer to thie [Esri Help Documentation](https://pro.arcgis.com/en/pro-app/arcpy/get-started/what-is-arcpy-.htm)

In [13]:
# Import Arcpy module
import arcpy

One important notes before analysis is to set the working environment for `Arcpy`. This is pretty similar to the concept of initializing the project folder when you are creating a new project in ArcGIS Pro or ArcMap. ArcGIS will automatically create a new file geodatabase for you and your analysis output will be stored in that filed geodatabase by default. 

In [14]:
# Set up your current working direectory
current_dir = r"C:\Users\user\Documents\ArcGIS\Projects"
# Set up your analysis workspace
arcpy.env.workspace = current_dir + r"\Assignment 7\Assignment 7.gdb"
# Allow file overwritting
arcpy.env.overwriteOutput = True
# Set the workspace outputCoordinateSystem environment
# This setting will automatically convert your analysis output layer to this projected system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(26915)

##### Display accidents based on their coordinates

In the cells below, we are going to plot accidents to a point feature class based on their coordinates information.

In [15]:
# Set path to input file and create output file name
# The input file is the csv file we created above
input_file = current_dir + r"\Assignment 7_script\Traffic Accidents in Metro.csv"
output_file = "Traffic_Accidents"

In [16]:
# Run the "XY Table To Point" tool
arcpy.management.XYTableToPoint(input_file, output_file, 
                                x_field = "YCOORD", y_field = "XCOORD")

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Traffic_Accidents'>

In [17]:
# Check the coordinate system
arcpy.Describe(output_file).SpatialReference.Name

'NAD_1983_UTM_Zone_15N'

##### Study Area Selection

The county shapefile covers the entire counties in the state of Minnesota, so we need to slice out counties in the Metro city

In [18]:
# Set up the path to county shapefile
study_area = current_dir + r"\Assignment 7\Data\shp_bdry_counties\County_Boundaries_in_Minnesota.shp"

In [19]:
# Read the fields containing in the shapefile and diplay their names and types
# Using the "ListFields" function
fieldlist = arcpy.ListFields(study_area)
for field in fieldlist:
    print(field.name,":", field.type)

FID : OID
Shape : Geometry
COUNTY_NAM : String
COUNTY_COD : String
COUNTY_FIP : String
COUNTY_GNI : Integer
ATP_CODE : String
SHAPE_Leng : Double
SHAPE_Area : Double


In [20]:
# Read the value of the "COUNTY_FIP" field to verify the fip code scheme
rows = arcpy.SearchCursor(study_area, fields = "COUNTY_FIP")
for row in rows:
    print("COUNTY_FIP: {0}".format(row.getValue("COUNTY_FIP")))
    break

COUNTY_FIP: 125


In [21]:
# Define SQL expression
where_clause = "{0} OR {1} OR {2} OR {3} OR {4} OR {5} OR {6}".format("COUNTY_FIP = '003'", 
                                                                      "COUNTY_FIP = '053'", 
                                                                      "COUNTY_FIP = '123'", 
                                                                      "COUNTY_FIP = '163'", 
                                                                      "COUNTY_FIP = '037'", 
                                                                      "COUNTY_FIP = '019'", 
                                                                      "COUNTY_FIP = '139'")

In [22]:
# Run the "Select Layers by Attribute" tool
Metro = arcpy.SelectLayerByAttribute_management(study_area, "NEW_SELECTION",
                                                where_clause)

In [23]:
# Export to a new feature class using the "Copy Features" tool
arcpy.CopyFeatures_management(Metro, 'Metro')

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Metro'>

In [24]:
# Check the coordinate system
arcpy.Describe("Metro").SpatialReference.Name

'NAD_1983_UTM_Zone_15N'

##### Cut accidents to the study area

In general, all accidents are supposed to be within our study area but there might be some accidents having coordinate errors. In order to exclude such potential errors, we can run the `Clip` tool to ensure all the accidents are in the study area.

In [25]:
# Set files names
inputfeature = "Traffic_Accidents"
outputfeature = "Traffic_Accidents_Metro"
cutter = "Metro"

In [26]:
# Run the "Clip" tool
arcpy.Clip_analysis(inputfeature, cutter, outputfeature)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Traffic_Accidents_Metro'>

Perfect!! Now we are all set. We are ready to run the "Spatial Join" tool and do some fun statistical analysis. 

### 2.2 Spatial Join

In this tutorial, we are tyring to answer some questions. The questions are as the following:
>1. Identify which county has the highest fatal number or injury number caused by accidents in 2016.
2. Identify which county has the highest rate of fatal or injury accidents in 2016.
3. Identify what accident type most frequently occured and which month and weekday had the highest accident occurence in each county.
4. Identify the location of accidents having the highest number of injuries or deaths.

We will not cover the `Spatial Join` tool explanation in this tutorial, for more information, please refer to this [Esri Help Documentation](https://pro.arcgis.com/en/pro-app/tool-reference/analysis/spatial-join.htm)

#### 2.2.1 Identify which county has the highest fatal number or injury number caused by accidents in 2016

To answer this question, we need to calculate total "fatal numbers" and "injury numbers" into the study area. Then, see which county has the highest fatal number and injury number and display its name and corresponding numbers. 

Two new fields, "Join_Count" and "TARGET_FID", are always added to the output feature class. "Join_Count" indicates how many join features match each target feature.


For this analysis, we will create **FieldMappings** object to help use interpret results easier. The **FieldMappings** object is a colection of FieldMap objects and it is used as the parameter value for tools that perform field maping, such as _Merge_ and _Spatial Join_. 

The procedure to work these objects with python scripts is as the following:

1. Create a **FieldMappings** object.
2. Initialize its **FieldMap** objects by adding the input feature classes or tables that are to be combined. 
3. Once all inputs are provided, the **FeildMap** object can be modified as you desired


- For more information about `FieldMap`, please refer to this [Esri Help Documentation](https://pro.arcgis.com/en/pro-app/arcpy/classes/fieldmap.htm)

In [27]:
# Define files' names
target_feature = "Metro"
join_feature = "Traffic_Accidents"
output_feature = "Fatal_or_Injury_Accidents_County"

In [28]:
# Creat the required FieldMap and FieldMappings objects
fm_injury = arcpy.FieldMap()
fm_fatal = arcpy.FieldMap()
fm_county = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get the field names of injury and death from original files
injury_accidents = "NUMINJ"
fatal_accidents = "NUMFAT"
county = "COUNTY_NAM"

# Add fields to their corresponding FieldMap objects
fm_injury.addInputField(join_feature, injury_accidents)
fm_fatal.addInputField(join_feature, fatal_accidents)
fm_county.addInputField(target_feature, county)

# Get the output field's properties as a field object
# Rename the field and pass the updated field object back into the field map
injury_name = fm_injury.outputField
injury_name.name = "Total Injury"
injury_name.aliasName = "Total Injury"
fm_injury.outputField = injury_name

fatal_name = fm_fatal.outputField
fatal_name.name = "Total Fatal"
fatal_name.aliasName = "Total Fatal"
fm_fatal.outputField = fatal_name

county_name = fm_county.outputField
county_name.name = "County"
county_name.aliasName = "County"
fm_county.outputField = county_name

# Set merge rule to each FieldMap object
# Note that merge rules only apply to attribute from the join features
# Do not apply merge rules to the target features' attribute
fm_injury.mergeRule = "Sum"
fm_fatal.mergeRule = "Sum"


# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_county)
fms.addFieldMap(fm_injury)
fms.addFieldMap(fm_fatal)

In [29]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Fatal_or_Injury_Accidents_County'>

In [30]:
# Let's check each field name and type
# Read the fields into a list and display field name and field type
fieldlist = arcpy.ListFields(output_feature)

for field in fieldlist:
    print(field.name,":", field.type)

OBJECTID : OID
Shape : Geometry
Join_Count : Integer
TARGET_FID : Integer
County : String
Total_Injury : Integer
Total_Fatal : Integer
Shape_Length : Double
Shape_Area : Double


In [31]:
# Using the "SearchCursor" function to iterate through the feature class and
# display the county name and injury number of the county has the highest injury number 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Total_Injury", 
                          sort_fields = "Total_Injury D")
for row in rows:
    print("{0} {1} {2}".format(
        row.getValue("County"),
        "county has the highest injury number casued by car accidents in 2016, the total count is",
        row.getValue("Total_Injury")))
    break

Hennepin county has the highest injury number casued by car accidents in 2016, the total count is 8112


In [32]:
# Using the "SearchCursor" function to iterate through the feature class and
# display the  name and fatal number of the county has the highest fatal number 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Total_Fatal", 
                          sort_fields = "Total_Fatal D")
for row in rows:
    print("{0} {1} {2}".format(
        row.getValue("County"),
        "county has the highest fatal number casued by car accidents in 2016, the total count is",
        row.getValue("Total_Fatal")))
    break

Hennepin county has the highest fatal number casued by car accidents in 2016, the total count is 47


#### 2.2.2 Identify which county has the highest rate of fatal or injury accidents in 2016

To identify which county has the highest ratio of fatal or injury accidents, we will first categorize accients into three types:
> 1. Accidents with deaths but no injuries
2. Accidents with injuries but no deaths
3. Accidents with injuries and deaths

Then, we will run three times spatial join to acquire a table containing **Total number of accidents**, **Total number of accidents with deaths**, **Total number of accidents with injuries**, and **Total number of accidents with injuries and deaths**. Based on these fields, we can caculate the rate of fatal or injury accidents.

In [33]:
# Define file's name
input_feature = "Traffic_Accidents_Metro"

##### Select out fatal accidents

In [34]:
# Define SQL expression
# 1 means Yes, 2 means no
where_clause = "Fatal = 1 AND INJURY = 2"

# Run the "Select Layer by Attribute tool" to select out accidents with deaths 
Fatal_accidents = arcpy.SelectLayerByAttribute_management(input_feature, "NEW_SELECTION", 
                                                          where_clause)

# Export to a new feature class using the "Copy Features" tool
arcpy.CopyFeatures_management(Fatal_accidents, 'Fatal_accidents')

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Fatal_accidents'>

##### Select out injury accidents

In [35]:
# Define SQL expression
# 1 means Yes, 2 means no
where_clause = "Fatal = 2 AND INJURY = 1"

# Run the "Select Layer by Attribute" tool to select out accidents with injuries
Injury_accidents = arcpy.SelectLayerByAttribute_management(input_feature, "NEW_SELECTION",
                                                where_clause)

# Export to a new feature class using the "Copy Features" tool
arcpy.CopyFeatures_management(Injury_accidents, 'Injury_accidents')

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_accidents'>

##### Select out fatal and injury accidents

In [36]:
# Define SQL expression
# 1 means Yes, 2 means no
where_clause = "Fatal = 1 AND INJURY = 1"

# Run the "Select Layer by Attribute" tool to select out accidents with injuries and deaths
Fatal_Injury_accidents = arcpy.SelectLayerByAttribute_management(input_feature,
                                                                 "NEW_SELECTION",
                                                                 where_clause)

# Export to a new feature class using the "Copy Features" tool
arcpy.CopyFeatures_management(Fatal_Injury_accidents, 'Fatal_Injury_accidents')

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Fatal_Injury_accidents'>

#### 2.2.3 Count the number of total accidents

- Since we already created a feature class containg total accidents in each county, we will not do that again and will use it as the first target feature.

###### First time spatial join

- Now, we are going to spatial join accidents with injuries to the layer containing total accident number 

In [37]:
# Define files' names
target_feature = "Total_Accidents_Metro"
join_feature = "Injury_accidents"
output_feature = "Injury_Accidents_Metro"

In [38]:
# Creat the required FieldMap and FieldMappings objects
fm_total = arcpy.FieldMap()
fm_county = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get the field names of county and total accidents from original files
total_accidents = "Join_Count"
county = "COUNTY_NAM"

# Add fields to their corresponding FieldMap objects
fm_total.addInputField(target_feature, total_accidents)
fm_county.addInputField(target_feature, county)

# Get the output field's properties as a field object
# Rename the field and pass the updated field object back into the field map
total_name = fm_total.outputField
total_name.name = "Total_Accidents"
total_name.aliasName = "Total_Accidents"
fm_total.outputField = total_name

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_county)
fms.addFieldMap(fm_total)

In [39]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_Accidents_Metro'>

###### Second time spatial join

- Spatial join accidents with deaths to the above output layer

In [40]:
# Define files' names
target_feature = "Injury_Accidents_Metro"
join_feature = "Fatal_accidents"
output_feature = "Injury_Fatal_Accidents_Metro"

In [41]:
# Create a new fieldmappings and add table of the target feature class
fms = arcpy.FieldMappings()
fms.addTable(target_feature)

# Get the Join_Count FieldMap through its index. 
# Join_Count is a field in target feature representing the number of accidents with injuries
injury_index = fms.findFieldMapIndex("Join_Count")
fm = fms.getFieldMap(injury_index)

# Get the output field's properties as a field object
injury = fm.outputField
# Rename the field and pass the updated field object back into the field map
injury.name = "Total_Injury"
injury.aliasName = "Total_Injury"
fm.outputField = injury

# Delete fields that are no longer applicable, such as TARGET_FID
x = fms.findFieldMapIndex("TARGET_FID")
fms.removeFieldMap(x)

# Replace the old fieldmap
fms.replaceFieldMap(injury_index, fm)

In [42]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_Fatal_Accidents_Metro'>

###### Third time spatial join

- Spatial join accidents with deaths and injuries to the above output layer

In [43]:
# Define files' names
target_feature = "Injury_Fatal_Accidents_Metro"
join_feature = "Fatal_Injury_accidents"
output_feature = "Injury_Fatal_I_F_Accidents_Metro"

In [44]:
# Create a new fieldmappings and add table of the target feature class
fms = arcpy.FieldMappings()
fms.addTable(target_feature)

# Get the Join_Count FieldMap through its index. 
# Join_Count is a field in target feature representing the number of accidents with deaths
fatal_index = fms.findFieldMapIndex("Join_Count")
fm = fms.getFieldMap(injury_index)

# Get the output field's properties as a field object
fatal = fm.outputField
# Rename the field and pass the updated field object back into the field map
fatal.name = "Total_Fatal"
fatal.aliasName = "Total_Fatal"
fm.outputField = fatal

# Delete fields that are no longer applicable, such as TARGET_FID
x = fms.findFieldMapIndex("TARGET_FID")
fms.removeFieldMap(x)

# Replace the old fieldmap
fms.replaceFieldMap(fatal_index, fm)

In [45]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_Fatal_I_F_Accidents_Metro'>

###### Rate calculation

Now our needed information are all set. Next, we will create a new field to calculate the rate.

In [46]:
# Define file's name
input_feature = "Injury_Fatal_I_F_Accidents_Metro"

In [47]:
# Define field's name and type
Field_name = "Rate"
Field_type = "Double"

# Run the Add Field tool 
arcpy.AddField_management(input_feature, Field_name, Field_type)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_Fatal_I_F_Accidents_Metro'>

In [48]:
# Print out all the fields' names in input feature
fields = arcpy.ListFields(input_feature)

for field in fields:
    print(field.name)

OBJECTID
Shape
Join_Count
TARGET_FID
Total_Fatal
Total_Injury
COUNTY_NAM
Total_Accidents
Shape_Length
Shape_Area
Rate


In [49]:
# Define expression
expression = "(!Join_Count! + !Total_Fatal! + !Total_Injury!) / !Total_Accidents!"

# Run the Field Calculator tool 
arcpy.CalculateField_management(input_feature, Field_name, expression)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Injury_Fatal_I_F_Accidents_Metro'>

In [50]:
# Using the "SearchCursor" function to iterate through the feature class and
# display the name and rate of the county has the highest rate 
rows = arcpy.SearchCursor(input_feature, 
                          fields = "County_NAM; Rate", 
                          sort_fields = "Rate D")
for row in rows:
    print("{0} {1} {2}".format(
        row.getValue("County_NAM"),
        "county has the highest ratio of injury and death accident in 2016, the rate is",
        row.getValue("Rate")))
    break

Anoka county has the highest ratio of injury and death accident in 2016, the rate is 0.3453800298062593


#### 2.2.4 Identify what accident type most frequently occured and which month and weekday had the highest accident occurence in each county.

To answer this question, we will include "Accident Type", "Weekday", and "Month" information to our output table so that we can see what accident type most frequently occured and which month and weekday had the highest accident occurence in each county.

In [51]:
# Define files' names
join_feature = "Traffic_Accidents_Metro"
target_feature = "Metro"
output_feature = "Mode_month_type_Metro"

In [52]:
# Creat the required FieldMap and FieldMappings objects
fm_month = arcpy.FieldMap()
fm_type = arcpy.FieldMap()
fm_county = arcpy.FieldMap()
fm_day = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get each field name from their original file
month = "MONTH"
atype = "ACCTYPE"
county = "COUNTY_NAM"
weekday = "ACCDAY"

# Add fields to their corresponding FieldMap objects
fm_month.addInputField(join_feature, month)
fm_type.addInputField(join_feature, atype)
fm_day.addInputField(join_feature, weekday)
fm_county.addInputField(target_feature, county)

# Get the output field's properties as a field object
# Rename the field and pass the updated field object back into the field map
month_name = fm_month.outputField
month_name.name = "Month"
month_name.aliasName = "Month"
fm_month.outputField = month_name 

type_name = fm_type.outputField
type_name.name = "Accident_type"
type_name.aliasName = "Accident_type"
fm_type.outputField = type_name

day_name = fm_day.outputField
day_name.name = "Accident_day"
day_name.aliasName = "Accident_day"
fm_day.outputField = day_name

county_name = fm_county.outputField
county_name.name = "County"
county_name.aliasName = "County"
fm_county.outputField = county_name

# Set merge rule to FieldMaps
fm_month.mergeRule = "Mode"
fm_type.mergeRule = "Mode"
fm_day.mergeRule = "Mode"

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_county)
fms.addFieldMap(fm_type)
fms.addFieldMap(fm_day)
fms.addFieldMap(fm_month)

In [53]:
# Run the "Spatial Join" tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, 
                           output_feature, match_option = "COMPLETELY_CONTAINS", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Mode_month_type_Metro'>

In [54]:
# Display fields' names of output feature
for field in arcpy.ListFields(output_feature):
    print(field.name)

OBJECTID
Shape
Join_Count
TARGET_FID
County
Accident_type
Accident_day
Month
Shape_Length
Shape_Area


In [55]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and prominent accident type in this county 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Accident_type")
for row in rows:
    print("{0} {1} {2} {3} county.".format(
        "Accident type:",
        row.getValue("Accident_type"),
        "occur most frequently in",
        row.getValue("County")))

Accident type: 10 occur most frequently in Hennepin county.
Accident type: 10 occur most frequently in Carver county.
Accident type: 10 occur most frequently in Ramsey county.
Accident type: 10 occur most frequently in Anoka county.
Accident type: 10 occur most frequently in Scott county.
Accident type: 10 occur most frequently in Washington county.
Accident type: 10 occur most frequently in Dakota county.


In [56]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and the month having the highest occurence of accidents in this county 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Month")
for row in rows:
    print("{0} {1} {2} {3} county.".format(
        "Month:",
        row.getValue("Month"),
        "has the highest occurence of accidents in",
        row.getValue("County")))

Month: 12 has the highest occurence of accidents in Hennepin county.
Month: 12 has the highest occurence of accidents in Carver county.
Month: 12 has the highest occurence of accidents in Ramsey county.
Month: 12 has the highest occurence of accidents in Anoka county.
Month: 12 has the highest occurence of accidents in Scott county.
Month: 12 has the highest occurence of accidents in Washington county.
Month: 12 has the highest occurence of accidents in Dakota county.


In [57]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and the weekday having the highest occurence of accidents in this county 
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Accident_day")
for row in rows:
    print("{0} {1} {2} {3} county.".format(
        "Weekday:",
        row.getValue("Accident_day"),
        "has the highest occurence of accidents in",
        row.getValue("County")))

Weekday: 3 has the highest occurence of accidents in Hennepin county.
Weekday: 4 has the highest occurence of accidents in Carver county.
Weekday: 6 has the highest occurence of accidents in Ramsey county.
Weekday: 6 has the highest occurence of accidents in Anoka county.
Weekday: 2 has the highest occurence of accidents in Scott county.
Weekday: 6 has the highest occurence of accidents in Washington county.
Weekday: 6 has the highest occurence of accidents in Dakota county.


#### 2.2.5 Identify the location of accidents having the highest number of injuries or deaths.

To answer this question, we will spatial join "Study Area" into "Accidents" in that county name information will be included into the output table so that we can know the county each accident locates in. 

In [58]:
# Define files' names
target_feature = "Traffic_Accidents_Metro"
join_feature = "Metro"
output_feature = "Traffic_Accidents_Locations"

In [59]:
# Creat the required FieldMap and FieldMappings objects
fm_county = arcpy.FieldMap()
fm_fatalnum = arcpy.FieldMap()
fm_injurynum = arcpy.FieldMap()
fms = arcpy.FieldMappings()

# Get each field name from their original file
county = "COUNTY_NAM"
fatalnum = "NUMFAT"
injurynum = "NUMINJ"

# Add fields to their corresponding FieldMap objects
fm_county.addInputField(join_feature, county)
fm_fatalnum.addInputField(target_feature, fatalnum)
fm_injurynum.addInputField(target_feature, injurynum)

# Get the output field's properties as a field object
# Rename the field and pass the updated field object back into the field map
county_name = fm_county.outputField
county_name.name = "County"
county_name.aliasName = "County"
fm_county.outputField = county_name

fatal_name = fm_fatalnum.outputField
fatal_name.name = "Fatal_Number"
county_name.aliasName = "Fatal_Number"
fm_fatalnum.outputField = fatal_name

injury_name = fm_injurynum.outputField
injury_name.name = "Injury_Number"
injury_name.aliasName = "Injury_Number"
fm_injurynum.outputField = injury_name

# Set merge rule to FieldMap
fm_county.mergeRule = "First"

# Add the FieldMap objects to the FieldMappings object
fms.addFieldMap(fm_county)
fms.addFieldMap(fm_fatalnum)
fms.addFieldMap(fm_injurynum)

In [60]:
# Run the Spatial Join tool with Fieldmappings object
arcpy.SpatialJoin_analysis(target_feature, join_feature, output_feature, 
                           join_operation = "JOIN_ONE_TO_MANY", match_option = "COMPLETELY_WITHIN", 
                           field_mapping = fms)

<Result 'C:\\Users\\user\\Documents\\ArcGIS\\Projects\\Assignment 7\\Assignment 7.gdb\\Traffic_Accidents_Locations'>

In [61]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and prominent accident type in this county 
# Since there might be accidents having the same number of deaths,
# we will print out the first five rows of records
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Fatal_Number D", 
                          sort_fields = "Fatal_Number D")

i = 0
for row in rows:
    print("{0} {1} {2} {3} county.".format(
        "Accidents with the highest number of deaths:",
        row.getValue("Fatal_Number"),
        "is located in",
        row.getValue("County")))
    i += 1
    if i >5:
        break

Accidents with the highest number of deaths: 3 is located in Dakota county.
Accidents with the highest number of deaths: 3 is located in Hennepin county.
Accidents with the highest number of deaths: 2 is located in Anoka county.
Accidents with the highest number of deaths: 2 is located in Dakota county.
Accidents with the highest number of deaths: 2 is located in Ramsey county.
Accidents with the highest number of deaths: 2 is located in Anoka county.


In [62]:
# Using the "SearchCursor" function to iterate through the feature class and
# display county name and prominent accident type in this county 
# Since there might be accidents having the same number of deaths,
# we will print out the first five rows of records
rows = arcpy.SearchCursor(output_feature, 
                          fields = "County; Injury_Number D", 
                          sort_fields = "Injury_Number D")

i = 0
for row in rows:
    print("{0} {1} {2} {3} county.".format(
        "Accidents with the highest number of injuries:",
        row.getValue("Injury_Number"),
        "is located in",
        row.getValue("County")))
    i += 1
    if i >5:
        break

Accidents with the highest number of injuries: 18 is located in Hennepin county.
Accidents with the highest number of injuries: 9 is located in Hennepin county.
Accidents with the highest number of injuries: 8 is located in Hennepin county.
Accidents with the highest number of injuries: 8 is located in Ramsey county.
Accidents with the highest number of injuries: 7 is located in Hennepin county.
Accidents with the highest number of injuries: 7 is located in Anoka county.


## 3. Conclusion

In this tutorial, we cover a wide range of topic including data cleanup, feature selection, spatial join, and result interpretation. After finishing this tutorial, you are expected to understand the advanced usage of the `Spatial Join` tool and how `Python` and `Arcpy` can aid in answering complex geospatial questions in an efficient and elegant manner. 

For more question about this tutorial, please email to lin00297@umn.edu.